## Sign language translation
#### ANN Network
#### Author : Gyan Krishna

In [2]:
from keras.layers import Dense, Flatten, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils, plot_model, to_categorical
from keras.callbacks import Callback
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np

from numpy import argmax
import pydot
import os

## Loading the data set

In [3]:
model_images_path = os.path.join(os.getcwd(),r'notebooks\model images')
accuracy_plots_path =os.path.join(os.getcwd(), r'notebooks\accuracy_plots')

npx_path = os.path.join(os.getcwd(), r"npx_dataset")
x_data = np.load(os.path.join(npx_path, "x_data.npy"))
y_data = np.load(os.path.join(npx_path, "y_data.npy"))

print(x_data.shape)
print(y_data.shape)

(87000, 100, 100)
(87000,)


## Splitting the data set 

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.15)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = 0.15)


print("x train shape      : ", x_train.shape)
print("y train shape      : ", y_train.shape)
print("x validation shape : ", x_valid.shape)
print("y validation shape : ", y_valid.shape)
print("x test shape       : ", x_test.shape)
print("y test shape       : ", y_test.shape)

x train shape      :  (62857, 100, 100)
y train shape      :  (62857,)
x validation shape :  (11093, 100, 100)
y validation shape :  (11093,)
x test shape       :  (13050, 100, 100)
y test shape       :  (13050,)


## Normalizing the data set


In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_valid = x_valid.astype('float32')

x_train /= 255
x_test /= 255
x_valid /= 255

## Converting y_test and y_train to categorical data

In [6]:
n_category = 29
y_train = np_utils.to_categorical(y_train, n_category)
y_test  = np_utils.to_categorical(y_test,  n_category)
y_valid = np_utils.to_categorical(y_valid,  n_category)

## Creating the base line model

In [7]:
def baseline_model():
    model = Sequential()
    model.add(Flatten(input_shape = (100, 100)))
    model.add(Dense(650, activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(350, activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(29))
    model.add(Activation('softmax'))

    model.summary()
    return model

In [8]:
model = baseline_model()
plot_model(model,
          to_file = os.path.join(accuracy_plots_path, "ann_baseline.jpg"),
          show_shapes = True,
          show_layer_names = True,
          rankdir = "TB",
          expand_nested = False,
          dpi = 150)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 10000)             0         
_________________________________________________________________
dense (Dense)                (None, 650)               6500650   
_________________________________________________________________
dropout (Dropout)            (None, 650)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 350)               227850    
_________________________________________________________________
dropout_1 (Dropout)          (None, 350)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 29)                10179     
_________________________________________________________________
activation (Activation)      (None, 29)                0

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\github\\American-Sign-Language-\\notebooks\\notebooks\\accuracy_plots\\ann_baseline.jpg'

In [ ]:
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [ ]:
epochs = 100
model_log = model.fit(x_train, y_train, 
                      batch_size = 256, 
                      epochs = 100, 
                      verbose = 1,
                      validation_data = (x_valid, y_valid))

In [ ]:
def evaluate_model(model, model_log, model_name):
    #print(model_log.history['loss'])
    print("train loss = ", model_log.history['loss'][-1])
    print("train accuracy = ", model_log.history['accuracy'][-1])
          
    print("test loss = ", model_log.history['val_loss'][-1])
    print("test accuracy = ", model_log.history['val_accuracy'][-1])

    plt.subplot(2,1,1)
    plt.title(model_name+' accuracy')
    plt.plot(range(1, epochs+1), model_log.history['accuracy'])
    plt.plot(range(1,  epochs+1), model_log.history['val_accuracy'])
    
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='lower right')
    
    plt.subplot(2,1,2)
    plt.title(model_name+' loss')
    plt.plot(model_log.history['loss'])
    plt.plot(model_log.history['val_loss'])
    
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['loss', 'test'], loc='lower right')
    
    plt.tight_layout()
    
    plt.savefig(model_name+os.path.join(model_images_path, "model.jpg"))
    return

In [ ]:
evaluate_model(model, model_log, "base line")